In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from collections import Counter
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import os
import re

In [3]:
def load_dataset(text):
    with open(text, 'r', encoding='utf-8') as f:
        data = f.read()
    return data

text = load_dataset("/content/Pride_and_Prejudice-Jane_Austen.txt")
print("Number of Characters = ", len(text))
print(text[:500])

Number of Characters =  711331
The Project Gutenberg eBook, Pride and Prejudice, by Jane Austen, Edited
by R. W. (Robert William) Chapman


This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org





Title: Pride and Prejudice


Author: Jane Austen

Editor: R. W. (Robert William) Chapman

Release Date: May 9, 2013  [eBook #42671]

Lang


In [4]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9.,!?;:'\" \n]", "", text)
    text = re.sub(r"\s+", " ", text)
    return list(text)

tokens = tokenize(text)
print("Total tokens = ", len(tokens))

Total tokens =  701900


In [5]:
def build_vocabulary(tokens):
    counter = Counter(tokens)
    vocabulary = set(counter.keys())
    stoi = {}
    itos = {}

    for i, token in enumerate(sorted(vocabulary)):
        stoi[token] = i+2
    stoi["<pad>"] = 0
    stoi["<unk>"] = 1

    for s, i in stoi.items():
        itos[i] = s

    return stoi, itos

stoi, itos = build_vocabulary(tokens)

In [6]:
def numericalize(tokens, stoi):
    indices = []
    for t in tokens:
        indices.append(stoi.get(t, stoi["<unk>"]))
    return indices

ids = numericalize(tokens, stoi)

In [7]:
def train_val_split(ids, val_ratio):
    split = int(len(ids) * (1-val_ratio))
    return ids[:split], ids[split:]

train_ids, val_ids = train_val_split(ids, val_ratio=0.3)

In [8]:
class LMDataset(Dataset):
    def __init__(self, token_ids, seq_len):
        self.token_ids = token_ids
        self.seq_len = seq_len

    def __len__(self):
        return len(self.token_ids)-self.seq_len

    def __getitem__(self, idx):
        x = torch.tensor(self.token_ids[idx:idx+self.seq_len])
        y = torch.tensor(self.token_ids[idx+1:idx+self.seq_len+1])

        return x,y

In [9]:
def data_loader(token_ids, seq_len, batch_size, shuffle):
    dataset = LMDataset(token_ids, seq_len)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return loader
seq_len = 64
batch_size = 32

train_loader = data_loader(train_ids, seq_len, batch_size, shuffle=True)
val_loader = data_loader(val_ids, seq_len, batch_size, shuffle=False)

In [10]:
class LSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.W = nn.Parameter(torch.randn(input_dim, 4 * hidden_dim)*0.1)
        self.U = nn.Parameter(torch.randn(hidden_dim, 4 * hidden_dim)*0.1)
        self.b = nn.Parameter(torch.zeros(4 * hidden_dim))

        for W in [self.W, self.U]:
            nn.init.xavier_uniform_(W)

    def forward(self, x, h, c):
        batch_size, seq_len, _ = x.size()
        hidden_outputs, cell_outputs = [], []

        for t in range(seq_len):
            x_t = x[:, t, :]
            gates = x_t @ self.W + h @ self.U + self.b
            forget_gate, input_gate, output_gate, activations = gates.chunk(4, dim=1)
            forget_gate = torch.sigmoid(forget_gate)
            input_gate = torch.sigmoid(input_gate)
            output_gate = torch.sigmoid(output_gate)
            activations = torch.tanh(activations)

            c = (forget_gate * c) + (input_gate * activations)
            h = torch.tanh(c) * output_gate

            hidden_outputs.append(h.unsqueeze(1))
            cell_outputs.append(c.unsqueeze(1))

        hidden_outputs = torch.cat(hidden_outputs, dim=1)
        cell_outputs = torch.cat(cell_outputs, dim=1)
        return hidden_outputs, cell_outputs

In [11]:
class MultiLayerLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(MultiLayerLSTM, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.layers = nn.ModuleList()
        self.layers.append(LSTMCell(input_dim, hidden_dim))
        for _ in range(1, num_layers):
            self.layers.append(LSTMCell(hidden_dim, hidden_dim))
        self.dropout = nn.Dropout(dropout)

        self.linear = nn.Linear(hidden_dim, input_dim)
        nn.init.xavier_normal_(self.linear.weight.data)
        self.linear.bias.data.fill_(0.0)

    def forward(self, x, h):
        hidden, cell = h
        output = x
        new_hidden, new_cell = [], []

        for layer_idx, layer in enumerate(self.layers):
            h_t, c_t = hidden[layer_idx], cell[layer_idx]
            output, cells = layer(output, h_t, c_t)
            new_hidden.append(output[:, -1].unsqueeze(0))
            new_cell.append(cells[:, -1].unsqueeze(0))
            output = self.dropout(output)
        return output, (torch.cat(new_hidden, dim=0), torch.cat(new_cell, dim=0))

In [12]:
class LSTMModel(nn.Module):
    def __init__(self, vocabulary_size, embed_dim, hidden_dim, num_layers, dropout):
        super().__init__()

        self.vocabulary_size = vocabulary_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(vocabulary_size, embed_dim)

        self.LSTM = MultiLayerLSTM(
            input_dim=embed_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            dropout=dropout
        )

        self.fc = nn.Linear(hidden_dim, vocabulary_size)
        nn.init.xavier_uniform_(self.fc.weight)

    def init_hidden(self, batch_size, device):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return (h0, c0)

    def count_parameters(self):
        total = 0
        for p in self.parameters():
            if p.requires_grad:
                total += p.numel()
        return total

    def forward(self, x, hidden=None):
        batch_size = x.size(0)
        if hidden is None:
            hidden = self.init_hidden(batch_size, x.device)
        x = self.embedding(x)
        lstm_output, hidden = self.LSTM(x, hidden)
        logits = self.fc(lstm_output)
        return logits, hidden

In [13]:
vocabulary_size = len(stoi)

model = LSTMModel(
    vocabulary_size=vocabulary_size,
    embed_dim=128,
    hidden_dim=256,
    num_layers=2,
    dropout=0.2
)

print("Model Parameters = ", model.count_parameters)
batch = next(iter(train_loader))[0]   # get x only
logits, hidden = model(batch)

print("Logits shape:", logits.shape)

Model Parameters =  <bound method LSTMModel.count_parameters of LSTMModel(
  (embedding): Embedding(47, 128)
  (LSTM): MultiLayerLSTM(
    (layers): ModuleList(
      (0-1): 2 x LSTMCell()
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (linear): Linear(in_features=256, out_features=128, bias=True)
  )
  (fc): Linear(in_features=256, out_features=47, bias=True)
)>
Logits shape: torch.Size([32, 64, 47])


In [14]:
def train_one_epoch(model, dataloader, criterion, optimizer, device, clip=1.0):
    model.train()
    total_loss = 0

    hidden = None  # LSTM hidden state

    tqdm_bar = tqdm(dataloader, desc="Training", leave=False)

    for x, y in tqdm_bar:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        if hidden is not None and hidden[0].size(1) != x.size(0):
            hidden = None

        # Forward pass
        logits, hidden = model(x, hidden)

        # Detach hidden state to prevent gradients from flowing through entire history
        if hidden is not None:
            if isinstance(hidden, tuple):  # LSTM
                hidden = tuple(h.detach() for h in hidden)
            else:  # GRU
                hidden = tuple(hidden.detach() for h in hidden)

        # Reshape for cross-entropy
        loss = criterion(
            logits.view(-1, logits.size(-1)),  # (B*L, vocab)
            y.view(-1)                         # (B*L)
        )

        # Backprop
        loss.backward()

        # Gradient clipping to prevent exploding gradients
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        total_loss += loss.item()

        avg_loss = total_loss / (tqdm_bar.n + 1)
        tqdm_bar.set_postfix(loss=f"{avg_loss:.2f}")

    return avg_loss

In [15]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0

    hidden = None

    with torch.no_grad():
        for x, y in tqdm(dataloader, desc="Validation", leave=False):
            x = x.to(device)
            y = y.to(device)

            if hidden is not None and hidden[0].size(1) != x.size(0):
                hidden = None

            logits, hidden = model(x, hidden)

            # Detach hidden
            if hidden is not None:
                if isinstance(hidden, tuple):
                    hidden = tuple(h.detach() for h in hidden)
                else:
                    hidden = tuple(hidden.detach() for h in hidden)

            loss = criterion(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )

            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss


In [16]:
def train_model(model, train_loader, val_loader, epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []
    perplexities = []

    model.to(device)

    for epoch in range(1, epochs + 1):
        print(f"\nEpoch {epoch}/{epochs}")

        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss   = evaluate(model, val_loader, criterion, device)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        perplexities.append(math.exp(val_loss))

        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss:   {val_loss:.4f}")
        print(f"  Val Perplexity: {math.exp(val_loss):.4f}")

    return train_losses, val_losses, perplexities


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LSTMModel(
    vocabulary_size=len(stoi),
    embed_dim=128,
    hidden_dim=256,
    num_layers=2,
    dropout=0.2
)

train_losses, val_losses, perplexities = train_model(
    model,
    train_loader,
    val_loader,
    epochs=10,
    lr=0.001,
    device=device
)

torch.save(model.state_dict(), "best_model.pt")


Epoch 1/10


  Train Loss: 1.1362
  Val Loss:   1.2568
  Val Perplexity: 3.5142

Epoch 2/10


  Train Loss: 0.9903
  Val Loss:   1.2838
  Val Perplexity: 3.6103

Epoch 3/10


  Train Loss: 0.9616
  Val Loss:   1.2990
  Val Perplexity: 3.6657

Epoch 4/10


  Train Loss: 0.9475
  Val Loss:   1.3031
  Val Perplexity: 3.6806

Epoch 5/10


...

  Train Loss: 0.9315
  Val Loss:   1.3188
  Val Perplexity: 3.7390

Epoch 7/10


  Train Loss: 0.9255
  Val Loss:   1.3220
  Val Perplexity: 3.7509

Epoch 8/10


  Train Loss: 0.9207
  Val Loss:   1.3242
  Val Perplexity: 3.7593

Epoch 9/10


  Train Loss: 0.9159
  Val Loss:   1.3232
  Val Perplexity: 3.7556

Epoch 10/10


Validation:  71%|███████   | 4666/6579 [01:59<01:02, 30.74it/s]

In [ ]:
os.makedirs("plots", exist_ok=True)
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(8,5))
plt.plot(epochs, train_losses, label="Train Loss", marker='o')
plt.plot(epochs, val_losses, label="Validation Loss", marker='o')
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("plots/loss_curve_128x256.png")
plt.show()

# 2️⃣ Perplexity Curve
plt.figure(figsize=(8,5))
plt.plot(epochs, perplexities, label="Validation Perplexity", color='purple', marker='o')
plt.title("Validation Perplexity over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.yscale("log")  # Optional: log-scale for better visualization
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.savefig("plots/perplexity_curve_128x256.png")
plt.show()

print("Plots saved.")